In [1]:
import os
import json
import pickle
import random
import numpy as np
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import label_binarize
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader, Dataset, Subset
import torchvision
from torchvision import transforms, datasets
import torch.backends.cudnn as cudnn
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
import optuna
from torch.cuda import amp

os.makedirs('/kaggle/working/artifacts', exist_ok=True)

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.benchmark = False
    cudnn.deterministic = True

set_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class AlbumentationsDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None, class_to_idx=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform
        self.class_to_idx = class_to_idx
        
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        image = Image.open(self.file_paths[idx]).convert('RGB')
        image = np.array(image)
        label = self.labels[idx]
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
            
        return image, label

def get_transforms():
    train_transform = A.Compose([
        A.Resize(224, 224),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.3),
        A.RandomBrightnessContrast(p=0.3),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])
    
    val_transform = A.Compose([
        A.Resize(224, 224),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])
    
    return train_transform, val_transform

def create_datasets():
    data_dir = '/kaggle/input/multiclass-skin-disease-dataset/Skin Lesion Dataset'
    train_dir = os.path.join(data_dir, 'train')
    val_dir = os.path.join(data_dir, 'val')
    test_dir = os.path.join(data_dir, 'test')
    
    train_dataset = datasets.ImageFolder(train_dir)
    val_dataset = datasets.ImageFolder(val_dir)
    test_dataset = datasets.ImageFolder(test_dir)
    
    class_names = train_dataset.classes
    class_to_idx = train_dataset.class_to_idx
    
    train_paths = [img[0] for img in train_dataset.imgs]
    train_labels = [img[1] for img in train_dataset.imgs]
    
    val_paths = [img[0] for img in val_dataset.imgs]
    val_labels = [img[1] for img in val_dataset.imgs]
    
    test_paths = [img[0] for img in test_dataset.imgs]
    test_labels = [img[1] for img in test_dataset.imgs]
    
    train_transform, val_transform = get_transforms()
    
    train_albumentations_dataset = AlbumentationsDataset(
        train_paths, train_labels, train_transform, class_to_idx
    )
    val_albumentations_dataset = AlbumentationsDataset(
        val_paths, val_labels, val_transform, class_to_idx
    )
    test_albumentations_dataset = AlbumentationsDataset(
        test_paths, test_labels, val_transform, class_to_idx
    )
    
    return (train_albumentations_dataset, val_albumentations_dataset, 
            test_albumentations_dataset, class_names)

class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='/kaggle/working/best_model.pth'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        
    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
            
    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

class BaselineModel(nn.Module):
    def __init__(self, num_classes=6):
        super(BaselineModel, self).__init__()
        self.backbone = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(in_features, num_classes)
        
    def forward(self, x):
        return self.backbone(x)

class HybridViTModel(nn.Module):
    def __init__(self, num_classes=6, transformer_depth=4, dropout=0.1):
        super(HybridViTModel, self).__init__()
        self.resnet = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-2])
        
        self.feature_dim = 2048
        self.patch_size = 7
        self.num_patches = (224 // self.patch_size) ** 2
        self.patch_embed_dim = 512
        
        self.patch_embed = nn.Conv2d(
            self.feature_dim, self.patch_embed_dim, 
            kernel_size=self.patch_size, stride=self.patch_size
        )
        
        self.cls_token = nn.Parameter(torch.zeros(1, 1, self.patch_embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches + 1, self.patch_embed_dim))
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.patch_embed_dim,
            nhead=8,
            dim_feedforward=1024,
            dropout=dropout,
            activation='gelu',
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=transformer_depth)
        
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.patch_embed_dim, num_classes)
        
        self._init_weights()
        
    def _init_weights(self):
        nn.init.normal_(self.cls_token, std=0.02)
        nn.init.normal_(self.pos_embed, std=0.02)
        nn.init.xavier_uniform_(self.patch_embed.weight)
        if self.patch_embed.bias is not None:
            nn.init.constant_(self.patch_embed.bias, 0)
            
    def forward(self, x):
        batch_size = x.size(0)
        
        features = self.resnet(x)
        
        patch_embeds = self.patch_embed(features)
        patch_embeds = patch_embeds.flatten(2).transpose(1, 2)
        
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, patch_embeds), dim=1)
        
        # Ensure positional embeddings match the sequence length
        if x.size(1) != self.pos_embed.size(1):
            pos_embed = self.pos_embed[:, :x.size(1), :]
        else:
            pos_embed = self.pos_embed
            
        x = x + pos_embed
        
        x = self.dropout(x)
        x = self.transformer(x)
        
        cls_output = x[:, 0]
        output = self.classifier(cls_output)
        
        return output

def train_epoch(model, dataloader, criterion, optimizer, device, scaler):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    
    pbar = tqdm(dataloader, desc='Training')
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        
        with torch.amp.autocast('cuda'):
            output = model(data)
            loss = criterion(output, target)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item()
        _, predicted = output.max(1)
        total_samples += target.size(0)
        correct_predictions += predicted.eq(target).sum().item()
        
        pbar.set_postfix({
            'Loss': f'{running_loss/(batch_idx+1):.4f}',
            'Acc': f'{100.*correct_predictions/total_samples:.2f}%'
        })
    
    epoch_loss = running_loss / len(dataloader)
    epoch_acc = 100. * correct_predictions / total_samples
    
    return epoch_loss, epoch_acc

def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    
    with torch.no_grad():
        for data, target in tqdm(dataloader, desc='Validation'):
            data, target = data.to(device), target.to(device)
            
            with torch.amp.autocast('cuda'):
                output = model(data)
                loss = criterion(output, target)
            
            running_loss += loss.item()
            _, predicted = output.max(1)
            total_samples += target.size(0)
            correct_predictions += predicted.eq(target).sum().item()
    
    epoch_loss = running_loss / len(dataloader)
    epoch_acc = 100. * correct_predictions / total_samples
    
    return epoch_loss, epoch_acc

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, early_stopping, epochs, device):
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
    scaler = torch.amp.GradScaler('cuda')
    
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device, scaler)
        val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
        
        if scheduler:
            scheduler.step()
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
        print(f'Learning Rate: {optimizer.param_groups[0]["lr"]:.6f}')
        print('-' * 50)
        
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
    
    return {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accs': train_accs,
        'val_accs': val_accs
    }

def objective(trial):
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)
    dropout = trial.suggest_float('dropout', 0.1, 0.5)
    transformer_depth = trial.suggest_int('transformer_depth', 2, 6)
    
    train_dataset, val_dataset, _, class_names = create_datasets()
    
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    model = HybridViTModel(
        num_classes=len(class_names),
        transformer_depth=transformer_depth,
        dropout=dropout
    ).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = CosineAnnealingLR(optimizer, T_max=10)
    
    early_stopping = EarlyStopping(patience=5, verbose=False)
    
    try:
        history = train_model(
            model, train_loader, val_loader, criterion, optimizer, 
            scheduler, early_stopping, epochs=10, device=device
        )
        best_val_acc = max(history['val_accs'])
    except RuntimeError as e:
        if 'out of memory' in str(e):
            torch.cuda.empty_cache()
            return float('inf')
        else:
            raise e
    
    return -best_val_acc

def evaluate_model(model, test_loader, device, class_names):
    model.eval()
    all_predictions = []
    all_targets = []
    all_probabilities = []
    
    with torch.no_grad():
        for data, target in tqdm(test_loader, desc='Testing'):
            data, target = data.to(device), target.to(device)
            
            output = model(data)
            probabilities = torch.softmax(output, dim=1)
            _, predicted = output.max(1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
    
    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)
    all_probabilities = np.array(all_probabilities)
    
    accuracy = 100. * (all_predictions == all_targets).sum() / len(all_targets)
    
    print(f'Test Accuracy: {accuracy:.2f}%')
    print('\nClassification Report:')
    print(classification_report(all_targets, all_predictions, target_names=class_names))
    
    cm = confusion_matrix(all_targets, all_predictions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig('/kaggle/working/artifacts/confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    y_test_bin = label_binarize(all_targets, classes=range(len(class_names)))
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    plt.figure(figsize=(10, 8))
    for i in range(len(class_names)):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], all_probabilities[:, i])
        roc_auc[i] = roc_auc_score(y_test_bin[:, i], all_probabilities[:, i])
        plt.plot(fpr[i], tpr[i], lw=2, 
                label=f'{class_names[i]} (AUC = {roc_auc[i]:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Multi-class ROC Curves')
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig('/kaggle/working/artifacts/roc_curves.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    metrics = {
        'accuracy': accuracy,
        'per_class_auc': {class_names[i]: float(roc_auc[i]) for i in range(len(class_names))},
        'macro_auc': float(roc_auc_score(y_test_bin, all_probabilities, average='macro')),
        'weighted_auc': float(roc_auc_score(y_test_bin, all_probabilities, average='weighted'))
    }
    
    return metrics, all_predictions, all_targets, all_probabilities

def create_streamlit_app(class_names):
    app_code = '''
import streamlit as st
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import json

class HybridViTModel(nn.Module):
    def __init__(self, num_classes=6, transformer_depth=4, dropout=0.1):
        super(HybridViTModel, self).__init__()
        from torchvision import models
        self.resnet = models.resnet50(weights=None)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-2])
        
        self.feature_dim = 2048
        self.patch_size = 7
        self.num_patches = (224 // self.patch_size) ** 2
        self.patch_embed_dim = 512
        
        self.patch_embed = nn.Conv2d(
            self.feature_dim, self.patch_embed_dim, 
            kernel_size=self.patch_size, stride=self.patch_size
        )
        
        self.cls_token = nn.Parameter(torch.zeros(1, 1, self.patch_embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches + 1, self.patch_embed_dim))
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.patch_embed_dim,
            nhead=8,
            dim_feedforward=1024,
            dropout=dropout,
            activation='gelu',
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=transformer_depth)
        
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.patch_embed_dim, num_classes)
        
    def forward(self, x):
        batch_size = x.size(0)
        features = self.resnet(x)
        patch_embeds = self.patch_embed(features)
        patch_embeds = patch_embeds.flatten(2).transpose(1, 2)
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, patch_embeds), dim=1)
        if x.size(1) != self.pos_embed.size(1):
            pos_embed = self.pos_embed[:, :x.size(1), :]
        else:
            pos_embed = self.pos_embed
        x = x + pos_embed
        x = self.dropout(x)
        x = self.transformer(x)
        cls_output = x[:, 0]
        output = self.classifier(cls_output)
        return output

@st.cache_resource
def load_model():
    model = HybridViTModel(num_classes=6)
    model.load_state_dict(torch.load('/kaggle/working/best_model.pth', map_location='cpu'))
    model.eval()
    return model

def preprocess_image(image):
    transform = A.Compose([
        A.Resize(224, 224),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])
    image = np.array(image.convert('RGB'))
    transformed = transform(image=image)
    return transformed['image'].unsqueeze(0)

st.set_page_config(page_title="Skin Disease Classifier", layout="wide")
st.title("🧬 Skin Disease Classification App")
st.write("Upload an image of a skin lesion to classify it into one of 6 categories.")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    col1, col2 = st.columns(2)
    
    with col1:
        st.image(image, caption="Uploaded Image", use_column_width=True)
    
    with col2:
        with st.spinner('Classifying...'):
            model = load_model()
            processed_image = preprocess_image(image)
            
            with torch.no_grad():
                outputs = model(processed_image)
                probabilities = torch.softmax(outputs, dim=1)
            
            probs = probabilities[0].numpy()
            class_names = ["Chickenpox", "Cowpox", "HFMD", "Healthy", "Measles", "Monkeypox"]
            
            top3_indices = np.argsort(probs)[-3:][::-1]
            top3_classes = [class_names[i] for i in top3_indices]
            top3_probs = [probs[i] for i in top3_indices]
            
            st.subheader("🔍 Prediction Results")
            st.success(f"**Predicted Class:** {top3_classes[0]} ({top3_probs[0]*100:.2f}%)")
            
            st.subheader("📊 Top 3 Predictions")
            for i, (cls, prob) in enumerate(zip(top3_classes, top3_probs)):
                st.write(f"{i+1}. {cls}: {prob*100:.2f}%")
            
            st.subheader("📈 All Class Probabilities")
            fig, ax = plt.subplots(figsize=(10, 6))
            y_pos = np.arange(len(class_names))
            ax.barh(y_pos, probs * 100, color='skyblue')
            ax.set_yticks(y_pos)
            ax.set_yticklabels(class_names)
            ax.set_xlabel('Probability (%)')
            ax.set_title('Class Probabilities')
            ax.invert_yaxis()
            st.pyplot(fig)

st.sidebar.header("ℹ️ Instructions")
st.sidebar.write("""
1. Upload a clear image of a skin lesion
2. Wait for the model to process the image
3. View the prediction results and probabilities
4. The app shows the top 3 most likely diagnoses

**Note:** This is for educational purposes only. 
Always consult a healthcare professional for medical diagnosis.
""")

st.sidebar.header("🏥 Disease Classes")
st.sidebar.write("""
- Chickenpox
- Cowpox  
- HFMD (Hand Foot Mouth Disease)
- Healthy
- Measles
- Monkeypox
""")
'''
    
    with open('/kaggle/working/streamlit_app.py', 'w') as f:
        f.write(app_code)

def main():
    print("Creating datasets...")
    train_dataset, val_dataset, test_dataset, class_names = create_datasets()
    
    print("Training baseline ResNet18 model...")
    baseline_model = BaselineModel(num_classes=len(class_names)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(baseline_model.parameters(), lr=1e-4, weight_decay=1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=20)
    early_stopping = EarlyStopping(patience=7, verbose=True)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
    
    baseline_history = train_model(
        baseline_model, train_loader, val_loader, criterion, optimizer, 
        scheduler, early_stopping, epochs=30, device=device
    )
    
    print("Running Optuna hyperparameter optimization...")
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=3, timeout=1200)
    
    if len(study.trials) > 0:
        print("Best hyperparameters:")
        for key, value in study.best_params.items():
            print(f"  {key}: {value}")
        
        best_params = study.best_params
    else:
        print("No successful trials. Using default parameters.")
        best_params = {'lr': 1e-4, 'weight_decay': 1e-4, 'dropout': 0.1, 'transformer_depth': 4}
    
    print("Training final hybrid model with best hyperparameters...")
    final_model = HybridViTModel(
        num_classes=len(class_names),
        transformer_depth=best_params['transformer_depth'],
        dropout=best_params['dropout']
    ).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(
        final_model.parameters(), 
        lr=best_params['lr'], 
        weight_decay=best_params['weight_decay']
    )
    scheduler = CosineAnnealingLR(optimizer, T_max=20)
    early_stopping = EarlyStopping(patience=10, verbose=True)
    
    final_history = train_model(
        final_model, train_loader, val_loader, criterion, optimizer,
        scheduler, early_stopping, epochs=30, device=device
    )
    
    print("Loading best model for evaluation...")
    final_model.load_state_dict(torch.load('/kaggle/working/best_model.pth'))
    
    print("Evaluating on test set...")
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
    metrics, predictions, targets, probabilities = evaluate_model(
        final_model, test_loader, device, class_names
    )
    
    print("Saving artifacts...")
    with open('/kaggle/working/artifacts/class_names.json', 'w') as f:
        json.dump(class_names, f)
    
    with open('/kaggle/working/artifacts/metrics.json', 'w') as f:
        json.dump(metrics, f, indent=2)
    
    train_transform, val_transform = get_transforms()
    with open('/kaggle/working/artifacts/transforms.pkl', 'wb') as f:
        pickle.dump({'train': train_transform, 'val': val_transform}, f)
    
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(final_history['train_losses'], label='Train Loss')
    plt.plot(final_history['val_losses'], label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    
    plt.subplot(1, 2, 2)
    plt.plot(final_history['train_accs'], label='Train Acc')
    plt.plot(final_history['val_accs'], label='Val Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.title('Training and Validation Accuracy')
    
    plt.tight_layout()
    plt.savefig('/kaggle/working/artifacts/training_history.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print("Creating Streamlit app...")
    create_streamlit_app(class_names)
    
    print("All tasks completed successfully!")
    print(f"Test Accuracy: {metrics['accuracy']:.2f}%")
    print(f"Macro AUC: {metrics['macro_auc']:.4f}")
    print("Artifacts saved to /kaggle/working/artifacts/")
    print("Streamlit app saved to /kaggle/working/streamlit_app.py")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Using device: cuda
Creating datasets...


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


Training baseline ResNet18 model...


100%|██████████| 44.7M/44.7M [00:00<00:00, 184MB/s]


...

Train Loss: 0.9361, Train Acc: 64.62%
Val Loss: 0.8450, Val Acc: 67.90%
Learning Rate: 0.000002
--------------------------------------------------
EarlyStopping counter: 10 out of 10
Early stopping triggered
Loading best model for evaluation...
Evaluating on test set...


Testing: 100%|██████████| 36/36 [00:05<00:00,  6.50it/s]


Test Accuracy: 99.29%

Classification Report:
              precision    recall  f1-score   support

  Chickenpox       0.97      0.98      0.98       113
      Cowpox       0.99      0.99      0.99        99
        HFMD       1.00      1.00      1.00       242
     Healthy       1.00      0.99      1.00       171
     Measles       1.00      0.99      0.99        83
   Monkeypox       0.99      1.00      0.99       426

    accuracy                           0.99      1134
   macro avg       0.99      0.99      0.99      1134
weighted avg       0.99      0.99      0.99      1134

Saving artifacts...


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Creating Streamlit app...
All tasks completed successfully!
Test Accuracy: 99.29%
Macro AUC: 0.9998
Artifacts saved to /kaggle/working/artifacts/
Streamlit app saved to /kaggle/working/streamlit_app.py


In [3]:
import os
import json
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                           classification_report, confusion_matrix, cohen_kappa_score,
                           matthews_corrcoef, log_loss, roc_auc_score, roc_curve, 
                           precision_recall_curve, auc)
from sklearn.preprocessing import label_binarize
from sklearn.manifold import TSNE
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms, datasets
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

os.makedirs('/kaggle/working/plots', exist_ok=True)
os.makedirs('/kaggle/working/misclassifications', exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class HybridViTModel(nn.Module):
    def __init__(self, num_classes=6, transformer_depth=4, dropout=0.1):
        super(HybridViTModel, self).__init__()
        self.resnet = torchvision.models.resnet50(weights=None)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-2])
        
        self.feature_dim = 2048
        self.patch_size = 7
        self.num_patches = (224 // self.patch_size) ** 2
        self.patch_embed_dim = 512
        
        self.patch_embed = nn.Conv2d(
            self.feature_dim, self.patch_embed_dim, 
            kernel_size=self.patch_size, stride=self.patch_size
        )
        
        self.cls_token = nn.Parameter(torch.zeros(1, 1, self.patch_embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches + 1, self.patch_embed_dim))
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.patch_embed_dim,
            nhead=8,
            dim_feedforward=1024,
            dropout=dropout,
            activation='gelu',
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=transformer_depth)
        
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.patch_embed_dim, num_classes)
        
        self.gradients = None
        self.activations = None
        
    def activations_hook(self, grad):
        self.gradients = grad
        
    def forward(self, x):
        batch_size = x.size(0)
        
        features = self.resnet(x)
        
        if self.training:
            h = features.register_hook(self.activations_hook)
        self.activations = features
        
        patch_embeds = self.patch_embed(features)
        patch_embeds = patch_embeds.flatten(2).transpose(1, 2)
        
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, patch_embeds), dim=1)
        
        if x.size(1) != self.pos_embed.size(1):
            pos_embed = self.pos_embed[:, :x.size(1), :]
        else:
            pos_embed = self.pos_embed
            
        x = x + pos_embed
        
        x = self.dropout(x)
        x = self.transformer(x)
        
        cls_output = x[:, 0]
        output = self.classifier(cls_output)
        
        return output
    
    def get_activations_gradient(self):
        return self.gradients
    
    def get_activations(self, x):
        return self.resnet(x)

class AlbumentationsDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        image = Image.open(self.file_paths[idx]).convert('RGB')
        image = np.array(image)
        label = self.labels[idx]
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
            
        return image, label, self.file_paths[idx]

def get_transforms():
    transform = A.Compose([
        A.Resize(224, 224),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])
    return transform

def load_model_and_data():
    class_names = ["Chickenpox", "Cowpox", "HFMD", "Healthy", "Measles", "Monkeypox"]
    
    model = HybridViTModel(num_classes=len(class_names))
    model.load_state_dict(torch.load('/kaggle/working/best_model.pth', map_location=device))
    model.to(device)
    model.eval()
    
    test_dir = '/kaggle/input/multiclass-skin-disease-dataset/Skin Lesion Dataset/test'
    test_dataset = datasets.ImageFolder(test_dir)
    
    test_paths = [img[0] for img in test_dataset.imgs]
    test_labels = [img[1] for img in test_dataset.imgs]
    
    transform = get_transforms()
    test_albumentations_dataset = AlbumentationsDataset(test_paths, test_labels, transform)
    
    test_loader = DataLoader(test_albumentations_dataset, batch_size=32, shuffle=False, num_workers=2)
    
    return model, test_loader, class_names, test_paths

def evaluate_model(model, test_loader, class_names):
    model.eval()
    all_predictions = []
    all_targets = []
    all_probabilities = []
    all_features = []
    all_file_paths = []
    
    with torch.no_grad():
        for data, target, file_paths in tqdm(test_loader, desc='Evaluating'):
            data, target = data.to(device), target.to(device)
            
            output = model(data)
            probabilities = torch.softmax(output, dim=1)
            _, predicted = output.max(1)
            
            features = model.get_activations(data)
            features = torch.nn.functional.adaptive_avg_pool2d(features, (1, 1))
            features = features.view(features.size(0), -1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
            all_features.extend(features.cpu().numpy())
            all_file_paths.extend(file_paths)
    
    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)
    all_probabilities = np.array(all_probabilities)
    all_features = np.array(all_features)
    
    accuracy = accuracy_score(all_targets, all_predictions)
    precision_macro = precision_score(all_targets, all_predictions, average='macro', zero_division=0)
    precision_micro = precision_score(all_targets, all_predictions, average='micro', zero_division=0)
    precision_weighted = precision_score(all_targets, all_predictions, average='weighted', zero_division=0)
    
    recall_macro = recall_score(all_targets, all_predictions, average='macro', zero_division=0)
    recall_micro = recall_score(all_targets, all_predictions, average='micro', zero_division=0)
    recall_weighted = recall_score(all_targets, all_predictions, average='weighted', zero_division=0)
    
    f1_macro = f1_score(all_targets, all_predictions, average='macro', zero_division=0)
    f1_micro = f1_score(all_targets, all_predictions, average='micro', zero_division=0)
    f1_weighted = f1_score(all_targets, all_predictions, average='weighted', zero_division=0)
    
    kappa = cohen_kappa_score(all_targets, all_predictions)
    mcc = matthews_corrcoef(all_targets, all_predictions)
    loss = log_loss(all_targets, all_probabilities)
    
    y_test_bin = label_binarize(all_targets, classes=range(len(class_names)))
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for i in range(len(class_names)):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], all_probabilities[:, i])
        roc_auc[i] = roc_auc_score(y_test_bin[:, i], all_probabilities[:, i])
    
    macro_auc = roc_auc_score(y_test_bin, all_probabilities, average='macro')
    micro_auc = roc_auc_score(y_test_bin, all_probabilities, average='micro')
    
    precision_dict = dict()
    recall_dict = dict()
    pr_auc = dict()
    
    for i in range(len(class_names)):
        precision_dict[i], recall_dict[i], _ = precision_recall_curve(y_test_bin[:, i], all_probabilities[:, i])
        pr_auc[i] = auc(recall_dict[i], precision_dict[i])
    
    per_class_accuracy = []
    for i in range(len(class_names)):
        class_mask = all_targets == i
        if np.sum(class_mask) > 0:
            class_acc = accuracy_score(all_targets[class_mask], all_predictions[class_mask])
            per_class_accuracy.append(class_acc)
        else:
            per_class_accuracy.append(0.0)
    
    metrics = {
        'accuracy': float(accuracy),
        'precision': {
            'macro': float(precision_macro),
            'micro': float(precision_micro),
            'weighted': float(precision_weighted)
        },
        'recall': {
            'macro': float(recall_macro),
            'micro': float(recall_micro),
            'weighted': float(recall_weighted)
        },
        'f1_score': {
            'macro': float(f1_macro),
            'micro': float(f1_micro),
            'weighted': float(f1_weighted)
        },
        'cohens_kappa': float(kappa),
        'matthews_correlation': float(mcc),
        'log_loss': float(loss),
        'auc': {
            'macro': float(macro_auc),
            'micro': float(micro_auc),
            'per_class': {class_names[i]: float(roc_auc[i]) for i in range(len(class_names))}
        },
        'per_class_accuracy': {class_names[i]: float(per_class_accuracy[i]) for i in range(len(class_names))}
    }
    
    return (metrics, all_predictions, all_targets, all_probabilities, 
            all_features, all_file_paths, fpr, tpr, roc_auc, precision_dict, recall_dict, pr_auc)

def generate_gradcam(model, image_tensor, target_class, class_names):
    model.eval()
    
    def backward_hook(module, grad_in, grad_out):
        model.gradients = grad_out[0]
    
    def forward_hook(module, input, output):
        model.activations = output
    
    handle_forward = model.resnet[-1].register_forward_hook(forward_hook)
    handle_backward = model.resnet[-1].register_backward_hook(backward_hook)
    
    image_tensor = image_tensor.unsqueeze(0).to(device)
    image_tensor.requires_grad_()
    
    output = model(image_tensor)
    model.zero_grad()
    
    one_hot_output = torch.zeros(output.size()).to(device)
    one_hot_output[0, target_class] = 1
    output.backward(gradient=one_hot_output)
    
    gradients = model.gradients
    activations = model.activations
    
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
    
    for i in range(activations.size(1)):
        activations[:, i, :, :] *= pooled_gradients[i]
    
    heatmap = torch.mean(activations, dim=1).squeeze()
    heatmap = torch.nn.functional.relu(heatmap)
    heatmap /= torch.max(heatmap)
    
    handle_forward.remove()
    handle_backward.remove()
    
    return heatmap.cpu().detach().numpy()

def plot_gradcam(model, test_loader, class_names, target_classes=['Monkeypox', 'Measles', 'Healthy']):
    model.eval()
    target_indices = [class_names.index(cls) for cls in target_classes]
    
    found_samples = {cls: None for cls in target_classes}
    
    for data, targets, file_paths in test_loader:
        data = data.to(device)
        targets = targets.cpu().numpy()
        
        for i, target in enumerate(targets):
            if class_names[target] in target_classes and found_samples[class_names[target]] is None:
                found_samples[class_names[target]] = (data[i], target, file_paths[i])
        
        if all(found_samples[cls] is not None for cls in target_classes):
            break
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    for idx, (cls_name, sample_data) in enumerate(found_samples.items()):
        if sample_data is None:
            continue
            
        image_tensor, true_class, file_path = sample_data
        class_idx = class_names.index(cls_name)
        
        heatmap = generate_gradcam(model, image_tensor, class_idx, class_names)
        
        original_image = Image.open(file_path).convert('RGB')
        original_image = original_image.resize((224, 224))
        original_img = np.array(original_image)
        
        heatmap_resized = cv2.resize(heatmap, (original_img.shape[1], original_img.shape[0]))
        heatmap_resized = np.uint8(255 * heatmap_resized)
        heatmap_colored = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)
        
        superimposed_img = heatmap_colored * 0.4 + original_img * 0.6
        superimposed_img = np.clip(superimposed_img, 0, 255).astype(np.uint8)
        
        axes[0, idx].imshow(original_img)
        axes[0, idx].set_title(f'Original: {cls_name}')
        axes[0, idx].axis('off')
        
        axes[1, idx].imshow(superimposed_img)
        axes[1, idx].set_title(f'Grad-CAM: {cls_name}')
        axes[1, idx].axis('off')
    
    plt.tight_layout()
    plt.savefig('/kaggle/working/plots/gradcam_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()

def plot_tsne(features, targets, class_names):
    tsne = TSNE(n_components=2, random_state=42, perplexity=30)
    features_2d = tsne.fit_transform(features)
    
    plt.figure(figsize=(12, 10))
    scatter = plt.scatter(features_2d[:, 0], features_2d[:, 1], c=targets, cmap='tab10', alpha=0.7)
    plt.colorbar(scatter)
    plt.title('t-SNE Visualization of Feature Embeddings')
    
    for i, class_name in enumerate(class_names):
        class_indices = np.where(targets == i)[0]
        if len(class_indices) > 0:
            centroid = np.mean(features_2d[class_indices], axis=0)
            plt.annotate(class_name, centroid, xytext=(5, 5), textcoords='offset points', 
                        fontsize=9, alpha=0.8)
    
    plt.tight_layout()
    plt.savefig('/kaggle/working/plots/tsne_features.png', dpi=300, bbox_inches='tight')
    plt.close()

def analyze_misclassifications(all_predictions, all_targets, all_file_paths, class_names):
    misclassified_indices = np.where(all_predictions != all_targets)[0]
    
    misclassification_data = []
    for idx in misclassified_indices:
        misclassification_data.append({
            'file_path': all_file_paths[idx],
            'true_label': class_names[all_targets[idx]],
            'predicted_label': class_names[all_predictions[idx]],
            'true_index': int(all_targets[idx]),
            'predicted_index': int(all_predictions[idx])
        })
    
    misclassification_df = pd.DataFrame(misclassification_data)
    
    plt.figure(figsize=(15, 12))
    num_samples = min(20, len(misclassified_indices))
    
    for i, idx in enumerate(misclassified_indices[:num_samples]):
        plt.subplot(4, 5, i + 1)
        
        image = Image.open(all_file_paths[idx])
        plt.imshow(image)
        plt.title(f'True: {class_names[all_targets[idx]]}\nPred: {class_names[all_predictions[idx]]}', 
                 fontsize=8)
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig('/kaggle/working/plots/misclassified_samples.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return misclassification_df

def create_visualizations(metrics, all_predictions, all_targets, all_probabilities, 
                         class_names, fpr, tpr, roc_auc, precision_dict, recall_dict, pr_auc):
    
    cm = confusion_matrix(all_targets, all_predictions)
    cm_normalized = confusion_matrix(all_targets, all_predictions, normalize='true')
    
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    plt.subplot(1, 2, 2)
    sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Normalized Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    plt.tight_layout()
    plt.savefig('/kaggle/working/plots/confusion_matrices.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    plt.figure(figsize=(10, 8))
    colors = plt.cm.Set3(np.linspace(0, 1, len(class_names)))
    
    for i, color in zip(range(len(class_names)), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                label=f'{class_names[i]} (AUC = {roc_auc[i]:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Multi-class ROC Curves')
    plt.legend(loc="lower right")
    plt.savefig('/kaggle/working/plots/roc_curve.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    plt.figure(figsize=(10, 8))
    for i, color in zip(range(len(class_names)), colors):
        plt.plot(recall_dict[i], precision_dict[i], color=color, lw=2,
                label=f'{class_names[i]} (AUC = {pr_auc[i]:.2f})')
    
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curves')
    plt.legend(loc="lower left")
    plt.savefig('/kaggle/working/plots/pr_curve.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    plt.figure(figsize=(10, 6))
    metrics_to_plot = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
    values = [metrics['accuracy'], metrics['precision']['macro'], 
              metrics['recall']['macro'], metrics['f1_score']['macro']]
    
    plt.bar(metrics_to_plot, values, color=['skyblue', 'lightcoral', 'lightgreen', 'gold'])
    plt.title('Key Performance Metrics')
    plt.ylabel('Score')
    plt.ylim(0, 1)
    
    for i, v in enumerate(values):
        plt.text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
    
    plt.savefig('/kaggle/working/plots/key_metrics.png', dpi=300, bbox_inches='tight')
    plt.close()

def main():
    print("Loading model and data...")
    model, test_loader, class_names, test_paths = load_model_and_data()
    
    print("Evaluating model...")
    (metrics, all_predictions, all_targets, all_probabilities, 
     all_features, all_file_paths, fpr, tpr, roc_auc, 
     precision_dict, recall_dict, pr_auc) = evaluate_model(model, test_loader, class_names)
    
    print("\n" + "="*50)
    print("EVALUATION RESULTS")
    print("="*50)
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Macro F1: {metrics['f1_score']['macro']:.4f}")
    print(f"Cohen's Kappa: {metrics['cohens_kappa']:.4f}")
    print(f"Macro AUC: {metrics['auc']['macro']:.4f}")
    print(f"Log Loss: {metrics['log_loss']:.4f}")
    
    print("\nClassification Report:")
    print(classification_report(all_targets, all_predictions, target_names=class_names))
    
    print("\nPer-class Accuracy:")
    for class_name, acc in metrics['per_class_accuracy'].items():
        print(f"{class_name}: {acc:.4f}")
    
    print("\nPer-class AUC:")
    for class_name, auc_val in metrics['auc']['per_class'].items():
        print(f"{class_name}: {auc_val:.4f}")
    
    print("\nGenerating visualizations...")
    create_visualizations(metrics, all_predictions, all_targets, all_probabilities, 
                         class_names, fpr, tpr, roc_auc, precision_dict, recall_dict, pr_auc)
    
    print("Generating Grad-CAM visualizations...")
    plot_gradcam(model, test_loader, class_names)
    
    print("Generating t-SNE visualization...")
    plot_tsne(all_features, all_targets, class_names)
    
    print("Analyzing misclassifications...")
    misclassification_df = analyze_misclassifications(all_predictions, all_targets, 
                                                     all_file_paths, class_names)
    
    print("Saving results...")
    with open('/kaggle/working/metrics.json', 'w') as f:
        json.dump(metrics, f, indent=2)
    
    misclassification_df.to_csv('/kaggle/working/misclassifications/misclassified_samples.csv', index=False)
    
    print("\n" + "="*50)
    print("EVALUATION COMPLETE")
    print("="*50)
    print(f"Results saved to:")
    print(f"- Metrics: /kaggle/working/metrics.json")
    print(f"- Plots: /kaggle/working/plots/")
    print(f"- Misclassifications: /kaggle/working/misclassifications/")
    print(f"Total misclassified samples: {len(misclassification_df)}")

if __name__ == "__main__":
    main()

Using device: cuda
Loading model and data...
Evaluating model...


Evaluating: 100%|██████████| 36/36 [00:09<00:00,  3.95it/s]



EVALUATION RESULTS
Accuracy: 0.9929
Macro F1: 0.9916
Cohen's Kappa: 0.9908
Macro AUC: 0.9998
Log Loss: 0.0226

Classification Report:
              precision    recall  f1-score   support

  Chickenpox       0.97      0.98      0.98       113
      Cowpox       0.99      0.99      0.99        99
        HFMD       1.00      1.00      1.00       242
     Healthy       1.00      0.99      1.00       171
     Measles       1.00      0.99      0.99        83
   Monkeypox       0.99      1.00      0.99       426

    accuracy                           0.99      1134
   macro avg       0.99      0.99      0.99      1134
weighted avg       0.99      0.99      0.99      1134


Per-class Accuracy:
Chickenpox: 0.9823
Cowpox: 0.9899
HFMD: 0.9959
Healthy: 0.9942
Measles: 0.9880
Monkeypox: 0.9953

Per-class AUC:
Chickenpox: 0.9992
Cowpox: 0.9999
HFMD: 1.0000
Healthy: 1.0000
Measles: 1.0000
Monkeypox: 0.9999

Generating visualizations...
Generating Grad-CAM visualizations...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Generating t-SNE visualization...
Analyzing misclassifications...
Saving results...

EVALUATION COMPLETE
Results saved to:
- Metrics: /kaggle/working/metrics.json
- Plots: /kaggle/working/plots/
- Misclassifications: /kaggle/working/misclassifications/
Total misclassified samples: 8
